In [1]:
# activate the project environment
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."));

  Activating project at `~/projects/julia-projects/dbscan`


In [2]:
using NearestNeighbors
using dbscan
using StaticArrays
using Dates
using CairoMakie
using LoggingExtras

In [3]:
fmt_logger = FormatLogger(stdout) do io, args
    # Write the module, level and message only
    println(io, args._module, " | ", "[", args.level, "] ", args.message)
end
debuglogger = MinLevelLogger(fmt_logger, Logging.Debug)

MinLevelLogger{FormatLogger, LogLevel}(FormatLogger(var"#11#12"(), VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(16) open, 0 bytes waiting)), VSCodeServer.io_send_callback), true), Debug)

In [4]:
with_logger(debuglogger) do
    @debug "test"
end

Main | [Debug] test


In [5]:
# points = [SVector{3}(rand(3)) for i in 1:1_000_000]
# points = map(i -> rand(3), 1:1_000_000)
# points = [rand(3) for _ in 1:1_000_000]
points = rand(SVector{2, Float64}, 1_000_000)

1000000-element Vector{SVector{2, Float64}}:
 [0.5042821870381974, 0.45060290167686723]
 [0.35925476462073536, 0.0008742008554812886]
 [0.14594993007568058, 0.5783671660335128]
 [0.25172110091830435, 0.2695602461836394]
 [0.13916936602793817, 0.08291187986567916]
 [0.6884718885455005, 0.8446891190363194]
 [0.3901174967484462, 0.06328391688093582]
 [0.5032220002307181, 0.28450398479264294]
 [0.31252242136590724, 0.031051325614298464]
 [0.2792451862167129, 0.8149933282070436]
 ⋮
 [0.6557352855746054, 0.727204119590567]
 [0.22145939697217554, 0.24520165996746546]
 [0.5358638131594505, 0.9588674972912131]
 [0.4851815453516828, 0.6726009523316184]
 [0.8861640912549457, 0.8459634949367411]
 [0.08116777865733926, 0.8782461755714192]
 [0.8265461437067768, 0.2639538223496041]
 [0.7350385968162959, 0.2612922725024981]
 [0.9400204007965723, 0.8131077188320192]

In [6]:
cluster_radius = 0.005
min_pts = 3

3

In [7]:
with_logger(debuglogger) do 
    clusters = dbscan.DBSCAN(points, cluster_radius, min_pts; n_threads = 1)
end

dbscan | [Debug] performing range searches
dbscan | [Debug] starting thread 1
dbscan | [Debug] thread 1 completed
dbscan | [Debug] merged 1 edge clusters
dbscan | [Debug] updating labels to cluster roots
dbscan | [Debug] DBSCAN completed


ValueIterator for a Dict{Int64, Vector{Int64}} with 1 entry. Values:
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  999991, 999992, 999993, 999994, 999995, 99…

In [8]:
clusters = with_logger(debuglogger) do 
    dbscan.DBSCAN(points, cluster_radius, min_pts; n_threads = 4)
end

dbscan | [Debug] performing range searches
dbscan | [Debug] starting thread 4
dbscan | [Debug] starting thread 1
dbscan | [Debug] starting thread 2
dbscan | [Debug] starting thread 3
dbscan | [Debug] thread 4 completed
dbscan | [Debug] thread 2 completed
dbscan | [Debug] thread 3 completed
dbscan | [Debug] thread 1 completed
dbscan | [Debug] merged 1 edge clusters
dbscan | [Debug] merged 2 edge clusters
dbscan | [Debug] merged 3 edge clusters
dbscan | [Debug] merged 4 edge clusters
dbscan | [Debug] updating labels to cluster roots
dbscan | [Debug] DBSCAN completed


ValueIterator for a Dict{Int64, Vector{Int64}} with 1 entry. Values:
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  999991, 999992, 999993, 999994, 999995, 99…

In [9]:
# for i in eachindex(clusters)
#     if clusters[i] != 0 
#         println(i, ' ', clusters[i])
#         j = dbscan.find_root(i, clusters)
#         clusters[i] = j
#     end
# end

In [10]:
# for i in eachindex(clusters)
#     if clusters[i] != 0 
#         println(i, ' ', clusters[i])
#     end
# end

In [11]:
minimum(map(length, clusters))

1000000

In [12]:
tree = KDTree(points)

KDTree{SVector{2, Float64}, Euclidean, Float64, SVector{2, Float64}}
  Number of points: 1000000
  Dimensions: 2
  Metric: Euclidean(0.0)
  Reordered: true

In [13]:
# inrange(tree, points[263613], cluster_radius)

In [14]:
function draw_circle!(ax, x0, y0, r)
    t = 0:0.01:2pi
    x = x0 .+ r .* cos.(t)
    y = y0 .+ r .* sin.(t)
    lines!(ax, x, y; color = :black, alpha = 0.1)
end

draw_circle! (generic function with 1 method)

In [15]:
# f = Figure()
# ax = Axis(f[1, 1])

# scatter!(points; color = :gray, markersize = 2.0)
# for (id, cluster) in enumerate(clusters)
#     scatter!(points[cluster])
#     for (x0, y0) in points[cluster]
#         draw_circle!(ax, x0, y0, cluster_radius)
#     end
# end

# f

In [26]:
# wrap timings in a function to avoid global variables causing problems
function run_tests(N; n_threads = 1)
    # times = zeros(N)
    times = []
    for i in 1:N
        points = rand(SVector{3, Float64}, 1_000_000)
        t0 = now()
        labels = dbscan.DBSCAN(points, 0.01, 3, n_threads = n_threads)
        tf = now()
        push!(times, canonicalize(tf - t0))
    end
    return times
end

run_tests (generic function with 1 method)

In [27]:
times_1 = run_tests(10; n_threads = 1)

10-element Vector{Any}:
 2 seconds, 879 milliseconds
 2 seconds, 899 milliseconds
 3 seconds, 53 milliseconds
 2 seconds, 964 milliseconds
 2 seconds, 970 milliseconds
 2 seconds, 974 milliseconds
 2 seconds, 859 milliseconds
 2 seconds, 746 milliseconds
 2 seconds, 936 milliseconds
 2 seconds, 862 milliseconds

In [28]:
times_2 = run_tests(10; n_threads = 2)

10-element Vector{Any}:
 2 seconds, 919 milliseconds
 3 seconds, 278 milliseconds
 3 seconds, 52 milliseconds
 3 seconds, 107 milliseconds
 3 seconds, 191 milliseconds
 3 seconds, 195 milliseconds
 3 seconds, 205 milliseconds
 3 seconds, 175 milliseconds
 3 seconds, 228 milliseconds
 3 seconds, 151 milliseconds

In [29]:
times_4 = run_tests(10; n_threads = 4)

10-element Vector{Any}:
 3 seconds, 481 milliseconds
 3 seconds, 487 milliseconds
 3 seconds, 462 milliseconds
 3 seconds, 449 milliseconds
 3 seconds, 511 milliseconds
 3 seconds, 331 milliseconds
 3 seconds, 585 milliseconds
 3 seconds, 358 milliseconds
 3 seconds, 374 milliseconds
 3 seconds, 607 milliseconds

In [20]:
# smaller dataset for plotting